# Create parquet files for analysis

## Imports

In [1]:
#!pip install xdem

In [1]:
import easysnowdata
import pystac_client
from sys import getsizeof
import tqdm
import planetary_computer
import adlfs
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import fsspec
import odc.stac
import pathlib
import configparser
import shapely
import time
import gc
import dask
import dask.distributed
import coiled
import matplotlib.pyplot as plt
import os
import traceback
import xdem
import rasterio
import hvplot.xarray
import seaborn as sns
import atexit
import dask.dataframe as dd


/home/eric/.local/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/home/eric/.local/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


## Config

In [ ]:
sas_token = pathlib.Path('sas_token.txt').read_text()

valid_tiles_gdf = gpd.read_file('valid_tiles.geojson')
valid_tiles_gdf = valid_tiles_gdf.sort_values(by='percent_valid_snow_pixels',ascending=False) 
#valid_tiles_gdf.explore(column='percent_valid_snow_pixels')

config = configparser.ConfigParser()

config.read('global_config.txt')
resolution = config.getfloat('VALUES', 'resolution')
zarr_chunk_size = (config.getint('VALUES', 'chunk_size'),config.getint('VALUES', 'chunk_size'))
bbox_left = config.getfloat('VALUES', 'bbox_left')
bbox_right = config.getfloat('VALUES', 'bbox_right')
bbox_top = config.getfloat('VALUES', 'bbox_top')
bbox_bottom = config.getfloat('VALUES', 'bbox_bottom')

WY_start = config.getint('VALUES', 'WY_start') 
WY_end = config.getint('VALUES', 'WY_end') 
water_years = np.arange(WY_start, WY_end + 1)

min_years_for_median_std = config.getint('VALUES', 'min_years_for_median_std')

min_monthly_acquisitions = config.getint('VALUES', 'min_monthly_acquisitions')
max_allowed_days_gap_per_orbit = config.getint('VALUES', 'max_allowed_days_gap_per_orbit')
low_backscatter_threshold = config.getfloat('VALUES', 'low_backscatter_threshold')

start_date = '2014-01-01'
today = pd.Timestamp.today().strftime('%Y-%m-%d')
end_date = today

print(f'Config loaded: \n{resolution=}, \n{zarr_chunk_size=}, \n{bbox_left=}, \n{bbox_right=}, \n{bbox_top=}, \n{bbox_bottom=}, \n{start_date=}, \n{end_date=} \n{water_years=}, \n{min_years_for_median_std=}, \n{low_backscatter_threshold=}, \n{min_monthly_acquisitions=}, \n{max_allowed_days_gap_per_orbit=}')

In [3]:
class Tile:
    def __init__(self, row, col):
        self.row = row
        self.col = col
        self.index = row,col
        self.percent_valid_snow_pixels = self.get_percent_valid_snow_pixels()
        self.geobox = self.get_geobox()
        self.bbox_gdf = self.get_bbox_gdf()
        self.start_time = None
        self.total_time = None
        self.s1_rtc_ds = None
        self.s1_rtc_ds_dims = None
        self.s1_rtc_masked_ds_dims = None
        self.runoff_onsets = None
        self.runoff_onsets_dims = None
        self.tr_2015 = None
        self.tr_2016 = None
        self.tr_2017 = None
        self.tr_2018 = None
        self.tr_2019 = None
        self.tr_2020 = None
        self.tr_2021 = None
        self.tr_2022 = None
        self.tr_2023 = None
        self.tr_2024 = None
        self.error_messages = []
        self.success = False

    
    def get_geobox(self):
        return geobox_tiles[self.index]
    
    def get_bbox_gdf(self):
        bbox = self.geobox.boundingbox
        bbox_geometry = shapely.geometry.box(bbox.left, bbox.bottom, bbox.right, bbox.top)
        bbox_gdf = gpd.GeoDataFrame(geometry=[bbox_geometry], crs=self.geobox.crs)
        return bbox_gdf
    
    def get_percent_valid_snow_pixels(self):
        return float(valid_tiles_gdf['percent_valid_snow_pixels'].loc[(valid_tiles_gdf['row'] == self.row) & (valid_tiles_gdf['col'] == self.col)].values[0])
    
    def get_info(self):
        return f"Processing Tile {self.index}"
    

In [4]:
valid_tiles_gdf = gpd.read_file('valid_tiles.geojson')
valid_tiles_gdf = valid_tiles_gdf.sort_values(by='percent_valid_snow_pixels',ascending=False) 

global_geobox = odc.geo.geobox.GeoBox.from_bbox((bbox_left, bbox_bottom,
   bbox_right, bbox_top),crs="epsg:4326", resolution=resolution)

geobox_tiles = odc.geo.geobox.GeoboxTiles(global_geobox,zarr_chunk_size)

tiles = [Tile(row,col) for row,col in zip(valid_tiles_gdf.row,valid_tiles_gdf.col)]

processed_tiles_df = pd.read_csv('tile_results.csv')
processed_tiles_df = processed_tiles_df[processed_tiles_df['success'] == True]
processed_tiles = set(zip(processed_tiles_df['row'], processed_tiles_df['col']))


tiles = [tile for tile in tiles if (tile.row, tile.col) in processed_tiles]

In [5]:
azure_filesystem = adlfs.AzureBlobFileSystem(account_name="snowmelt", credential=sas_token)

## Read global product

In [7]:
global_store = azure_filesystem.get_mapper("snowmelt/snowmelt_runoff_onset/global.zarr")
global_ds = xr.open_zarr(global_store, consolidated=True,decode_coords='all')

## Clip global product and reproject to UTM zone 

In [8]:
def add_coordinate_arrays(ds):
    """
    Create additional arrays to store original lat/lon coordinates.
    """
    lat, lon = np.meshgrid(ds.latitude, ds.longitude)
    ds['original_lat'] = xr.DataArray(lat.T, dims=('latitude', 'longitude'))
    ds['original_lon'] = xr.DataArray(lon.T, dims=('latitude', 'longitude'))
    return ds

## Rearrange data variables

In [9]:
def convert_water_year_dim_to_var(ds):
    for year in ds.water_year.values:
        ds[f'runoff_onset_WY{year}'] = ds['runoff_onset'].sel(water_year=year)

    ds = ds.drop_vars('runoff_onset').drop_vars('water_year')
    return ds

## Bring in variables: elevation, aspect, slope, tpi, snow class, esa worldcover, forest cover fraction... 

In [10]:
def add_topography(tile,ds):
    catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",modifier=planetary_computer.sign_inplace)
    search = catalog.search(collections=[f"cop-dem-glo-30"],intersects=tile.geobox.geographic_extent)
    dem_da = odc.stac.load(items=search.items(),like=ds,chunks={},resampling='bilinear')['data'].squeeze()
    dem_da = dem_da.rio.write_nodata(-32767,encoded=True).drop_vars('time').compute() # compute for xdem stuff

    ds['dem'] = dem_da

    # [xDEM](https://xdem.readthedocs.io/en/stable/index.html) to calculate slope and aspect and topographic position index

    attributes = xdem.terrain.get_terrain_attribute(
        ds['dem'],
        resolution=ds['dem'].rio.resolution()[0],
        attribute=["aspect", "slope", "topographic_position_index"],
    )

    ds['aspect'] = xr.DataArray(attributes[0], dims=ds['dem'].dims, coords=ds['dem'].coords)
    ds['slope'] = xr.DataArray(attributes[1], dims=ds['dem'].dims, coords=ds['dem'].coords)
    # TPI? https://xdem.readthedocs.io/en/stable/gen_modules/xdem.DEM.topographic_position_index.html, https://tc.copernicus.org/articles/8/1989/2014/tc-8-1989-2014.pdf
    # maybe incorrect radius...
    ds['tpi'] = xr.DataArray(attributes[2], dims=ds['dem'].dims, coords=ds['dem'].coords)

    # DAH?
    # alpha_max = 202.5 #only in northern hemisphere at specific latitude?
    # DAH_da = np.cos(np.deg2rad(alpha_max-aspect_da))*np.arctan(np.deg2rad(slope_da))

    return ds

def add_snow_class(tile,ds):

    snow_classification = easysnowdata.remote_sensing.get_seasonal_snow_classification(tile.bbox_gdf)
    ds['snow_classification'] = snow_classification.rio.reproject_match(ds['dem'],resampling=rasterio.enums.Resampling.mode)

    return ds

def add_esa_worldcover(tile,ds):

    esa_worldcover = easysnowdata.remote_sensing.get_esa_worldcover(tile.bbox_gdf)
    esa_worldcover.rio.set_nodata(0,inplace=True)
    ds['esa_worldcover'] = esa_worldcover.rio.reproject_match(ds['dem'],resampling=rasterio.enums.Resampling.mode)

    return ds

def add_forest_cover(tile,ds):

    forest_cover_fraction = easysnowdata.remote_sensing.get_forest_cover_fraction(tile.bbox_gdf)
    ds['forest_cover_fraction'] = forest_cover_fraction.rio.reproject_match(ds['dem'],resampling=rasterio.enums.Resampling.bilinear)

    return ds

In [11]:
# recalc median, std, nmad, trend, trend strength (use diff thresholds?)

## Convert dataset with all variables to dataframe where each pixel location is a row

In [12]:
def dataset_to_dataframe(tile,ds):
    # only drop row values / a pixel if there are no runoff_onset_predictions for any of the years
    # drop_subset = [f'runoff_onset_WY{water_year}' for water_year in water_years]

    #df = ds.to_dataframe().reset_index().dropna(subset=drop_subset,how='all').drop_vars('spatial_ref', axis=1) use this if we want to keep pixels with runoff_onset_predictions for two or less years, for three or more use next line
    df = ds.to_dataframe().reset_index().dropna(subset='runoff_onset_median').drop('spatial_ref', axis=1)

    df['tile_row'] = tile.row
    df['tile_col'] = tile.col
    hemisphere = 'northern' if ds.rio.crs.to_epsg() < 32700 else 'southern'
    df['hemisphere'] = hemisphere
    # change these as we include/exclude new vars
    df = df[["tile_row","tile_col","hemisphere","original_lat","original_lon","runoff_onset_WY2015","runoff_onset_WY2016","runoff_onset_WY2017","runoff_onset_WY2018","runoff_onset_WY2019","runoff_onset_WY2020","runoff_onset_WY2021","runoff_onset_WY2022","runoff_onset_WY2023","runoff_onset_WY2024","runoff_onset_median","runoff_onset_std","dem","aspect","slope","tpi","snow_classification","esa_worldcover","forest_cover_fraction"]]
    
    columns_to_round = [col for col in df.columns if col not in ['original_lat', 'original_lon', 'runoff_onset_std', 'hemisphere']]
    df[columns_to_round] = df[columns_to_round].replace([np.inf, -np.inf, np.nan], -9999)
    df[columns_to_round] = df[columns_to_round].round().astype(int)

    df['original_lat'] = df['original_lat'].round(4)
    df['original_lon'] = df['original_lon'].round(4)
    df['runoff_onset_std'] = df['runoff_onset_std'].round(2)

    for col in df.columns:
        if df[col].dtype == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif df[col].dtype == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
    
    return df

## Put it all together

In [13]:
def create_utm_datacube(tile, global_ds):
    tile_ds = global_ds.rio.clip_box(*tile.get_geobox().boundingbox,crs='EPSG:4326')
    tile_ds = add_coordinate_arrays(tile_ds)
    utm_crs = tile_ds.rio.estimate_utm_crs()
    tile_utm_ds = tile_ds.rio.reproject(utm_crs,resolution=80,resampling=rasterio.enums.Resampling.bilinear).persist()
    tile_utm_ds = convert_water_year_dim_to_var(tile_utm_ds)
    tile_utm_ds = add_topography(tile,tile_utm_ds)
    tile_utm_ds = add_snow_class(tile,tile_utm_ds)
    tile_utm_ds = add_esa_worldcover(tile,tile_utm_ds)
    tile_utm_ds = add_forest_cover(tile,tile_utm_ds)
    return tile_utm_ds

In [14]:
def create_and_save_analysis_parquet(tile, filename, filesystem, global_ds):
    print(f"Processing {filename}")
    try:
        tile_utm_ds = create_utm_datacube(tile, global_ds)
        tile_utm_df = dataset_to_dataframe(tile, tile_utm_ds)
        tile_utm_df.to_parquet(f"snowmelt/analysis/tiles/{filename}",filesystem=filesystem)
        print(f"Saved {filename}")
        return filename, True
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return filename, False

## test it out

In [ ]:
tile = tiles[2]
tiles[2].index
tile.geobox.explore(tiles='EsriWorldImagery')

In [ ]:
tile_utm_ds = create_utm_datacube(tile, global_ds)
tile_utm_ds

In [ ]:
tile_utm_df = dataset_to_dataframe(tile,tile_utm_ds)
tile_utm_df

## Write to files

In [ ]:
cluster = coiled.Cluster(idle_timeout="10 minutes",
                         n_workers=5,
                         worker_memory="64 GB",
                         worker_cpu=8,
                         scheduler_memory="32 GB",
                         spot_policy="spot",
                         environ={"GDAL_DISABLE_READDIR_ON_OPEN": "EMPTY_DIR"},
                         workspace="azure",
                         )

client = cluster.get_client()

In [ ]:
try:
    existing_filenames = [pathlib.Path(path).name for path in azure_filesystem.ls('snowmelt/analysis/tiles/')]
except:
    existing_filenames = []
        
futures = []

for tile in tqdm.tqdm(tiles,total=len(tiles)):

    filename = f'tile_{tile.row:03d}_{tile.col:03d}.parquet'

    if filename in existing_filenames:
        print(f'{filename} already exists, skipping...')
        continue

    future = client.submit(create_and_save_analysis_parquet, tile, filename, azure_filesystem, global_ds, retries=1)
    futures.append(future)

In [ ]:
for future, result in dask.distributed.as_completed(futures, with_results=True):
    print(f"{result[0]} : {'SUCCESS' if result[1] == True else 'FAILED'}")

In [ ]:
parquet_files = azure_filesystem.ls('snowmelt/analysis/tiles/')
parquet_files

## Code graveyard

In [ ]:
#view_tile(Tile(13,126)) very dense norway
#view_tile(Tile(1,133)) svalbard
#view_tile(Tile(88,72)) SA
#Tile(88,72).get_geobox().boundingbox
#test_ds = global_ds.rio.clip_box(-75,-51,-72,-48,crs='EPSG:4326') area surrounding SA tile, other tiles should be adjacenent
#f,ax=plt.subplots(2,1,figsize=(10,10))
#test_ds['runoff_onset_median'].plot.imshow(ax=ax[0],vmin=0,vmax=365)
#test_ds['runoff_onset_std'].plot.imshow(ax=ax[1],cmap='Reds')
#test_ds['runoff_onset'].plot.imshow(col='water_year',col_wrap=3,vmin=0,vmax=365)


# def view_tile(tile: Tile):


#     test_ds = global_ds.rio.clip_box(*tile.get_geobox().boundingbox,crs='EPSG:4326')

#     f,ax=plt.subplots(2,1,figsize=(10,10))
#     test_ds['runoff_onset_median'].plot.imshow(ax=ax[0],vmin=0,vmax=365)

#     test_ds['runoff_onset_std'].plot.imshow(ax=ax[1],cmap='Reds')

#     test_ds['runoff_onset'].plot.imshow(col='water_year',col_wrap=3,vmin=0,vmax=365)

In [ ]:
# tile_ds = global_ds.rio.clip_box(*tile.get_geobox().boundingbox,crs='EPSG:4326').compute()
# tile_ds

# tile_ds = add_coordinate_arrays(tile_ds)
# tile_ds
# utm_crs = tile_ds.rio.estimate_utm_crs()
# tile_utm_ds = tile_ds.rio.reproject(utm_crs,resolution=80,resampling=rasterio.enums.Resampling.bilinear)
# tile_utm_ds

# tile_utm_ds['runoff_onset'].plot.imshow(col='water_year',col_wrap=3,robust=True)

# f,axs = plt.subplots(1,2,figsize=(10,5))

# tile_utm_ds['runoff_onset_median'].plot.imshow(ax=axs[0],robust=True)
# tile_utm_ds['runoff_onset_std'].plot.imshow(ax=axs[1],robust=True,cmap='Reds')

# for ax in axs:
#     ax.set_aspect('equal')

# f,axs=plt.subplots(1,2,figsize=(10,5))
# tile_utm_ds['original_lat'].plot.imshow(ax=axs[0])
# tile_utm_ds['original_lon'].plot.imshow(ax=axs[1])
# axs[0].set_title('original_lat')
# axs[1].set_title('original_lon')

# for ax in axs:
#     ax.set_aspect('equal')

# tile_utm_ds = convert_water_year_dim_to_var(tile_utm_ds)
# tile_utm_ds
# tile_utm_ds = add_topography(tile,tile_utm_ds)
# tile_utm_ds
# tile_utm_ds = add_snow_class(tile,tile_utm_ds)
# tile_utm_ds
# tile_utm_ds = add_esa_worldcover(tile,tile_utm_ds)
# tile_utm_ds
# tile_utm_ds = add_forest_cover(tile,tile_utm_ds)
# tile_utm_ds
# tile_utm_df = dataset_to_dataframe(tile_utm_ds,utm_crs,water_years)
# tile_utm_df
# tile_results_df = pd.read_csv(f'results/tile_{tile.row:03d}_{tile.col:03d}.csv')


# var_list = ['runoff_onset_median','runoff_onset_std','aspect','slope','tpi','snow_classification','esa_worldcover','forest_cover_fraction']
# tile_utm_ds.hvplot.image(z=var_list,tiles="EsriImagery",crs=tile_utm_ds.rio.crs,width=500,height=500) # hover_cols=var_list


# num_vars = len(tile_utm_ds.data_vars)

# # Calculate the number of rows and columns for the subplots
# num_cols = 3  # You can adjust this
# num_rows = (num_vars + num_cols - 1) // num_cols

# # Create a figure with subplots
# fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5*num_rows))
# axes = axes.flatten()  # Flatten the axes array for easy indexing

# # Loop through each variable and plot
# for i, (var_name, da) in enumerate(tile_utm_ds.data_vars.items()):
#     ax = axes[i]
#     da.plot(ax=ax)
#     ax.set_title(var_name)

# # Remove any unused subplots
# for j in range(i+1, len(axes)):
#     fig.delaxes(axes[j])

# # f,axs=plt.subplots(2,2,figsize=(10,10),sharex=True,sharey=True)


# # tile_utm_ds['dem'].plot.imshow(ax=axs[0,0])
# # tile_utm_ds['aspect'].plot.imshow(ax=axs[0,1],cmap='twilight')
# # tile_utm_ds['slope'].plot.imshow(ax=axs[1,0],cmap='Reds')
# # tile_utm_ds['tpi'].plot.imshow(ax=axs[1,1],cmap='Purples')

# # titles = ['DEM','Aspect','Slope','TPI']

# # for ax,title in zip(axs.flatten(),titles):
# #     ax.set_aspect('equal')
# #     ax.set_title(title)

# # f.tight_layout()
# # 
# # #hvplot.explorer(tile_utm_ds, x='x', y='y')
# # 
# # #hvplot.help("image")

# tile_utm_df.to_csv(f'results/tile_{tile.row:03d}_{tile.col:03d}.csv',index=False)